## Matrizes e Vetor de Cargas

In [2]:
import numpy as np
import scipy.linalg as sc
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import pandas as pd
from pandas import ExcelWriter

#-------------------------------------------------------------------------    
#1. Dados de Entrada  
#-------------------------------------------------------------------------  

nos = pd.read_excel('dados_de_entradaj.xlsx','nos')
barras = pd.read_excel('dados_de_entradaj.xlsx','barras')

#-------------------------------------------------------------------------     
#2. Vetores auxiliares 
#------------------------------------------------------------------------- 

n_id = list(nos['Nó'])          # Identificação dos Nós
nn   = len(list(nos['Cx']))     # número de nós
ngl  = len(list(nos['Cx']))*3   # número de graus de liberdade 
u    = list(nos['u'])[0:nn]     # Identificação do Grau de Liberdade
v    = list(nos['v'])[0:nn]     # Identificação do Grau de Liberdade
teta = list(nos['teta'])[0:nn]  # Identificação do Grau de Liberdade

#2.1 Lista dos graus de liberdade

a_gls = []

for i in range (nn):
    gls_i  = [u[i],v[i],teta[i]]
    a_gls.append(gls_i)

#-------------------------------------------------------------------------     
#3. Vetor de coordenadas ( X e Y dos nós) 
#------------------------------------------------------------------------- 

cx   = list(nos['Cx'])[0:nn]    # coordenada X
cy   = list(nos['Cy'])[0:nn]    # coordenada Y
cz   = list(nos['Cz'])[0:nn]    # coordenada Z

#-------------------------------------------------------------------------     
#4. Matriz de identificação dos nós( Nó inicial (N) e Nó Final (F) de cada barra)
#-------------------------------------------------------------------------  

noN     = list(barras['noN'])  #Nós Iniciais
noF     = list(barras['noF'])  #Nós Finais
nb      = len(noN)             #Número de Barras
IDN      = np.zeros((2,nb))
IDN[0,:] = noN
IDN[1,:] = noF

#-------------------------------------------------------------------------     
#5. Propriedades de cada barra
#-------------------------------------------------------------------------  

A   = list(barras['Area(m2)'])
I   = list(barras['Inérciaz(m4)'])
E   = 2.1*10**8                    #[kN/m2]

#------------------------------------------------------------------------- 
#6. Matriz de identificação das Barras em relação aos Graus de Liberdade
#-------------------------------------------------------------------------  

IDB = np.zeros((6,nb)) #Graus de liberdade por nó, número de barras

for i in range(3):

    IDB[i,:]   = IDN[0,:]*3-2+i
    IDB[i+3,:] = IDN[1,:]*3-2+i

#-------------------------------------------------------------------------        
#7. Comprimento de cada barra e cossenos diretores
#-------------------------------------------------------------------------  

Lx   = np.zeros(nb)
Ly   = np.zeros(nb)
cosx = np.zeros(nb)
cosy = np.zeros(nb)
L    = np.zeros(nb)

for n in range (nb):

    k1      = int(IDN[0,n] -1)  # Indexador da matriz IDN
    k2      = int(IDN[1,n] -1)  # Indexador da matriz IDN
    Lx[n]   = cx[k2] - cx[k1]
    Ly[n]   = cy[k2] - cy[k1]
    L[n]    = np.sqrt(Lx[n]**2 + Ly[n]**2)
    cosx[n] = Lx[n]/L[n]
    cosy[n] = Ly[n]/L[n]
        
#-------------------------------------------------------------------------        
#8. Matriz de rigidez
#-------------------------------------------------------------------------  

K = np.zeros((ngl,ngl))                

for i in range (nb):   
    
#8.1 Matriz de rigidez local da barra
   
    k = np.array([[E*A[i]/L[i], 0, 0, -E*A[i]/L[i],0 ,0 ],
                  [0, 12*E*I[i]/(L[i]**3), 6*E*I[i]/(L[i]**2), 0, -12*E*I[i]/(L[i]**3),6*E*I[i]/(L[i]**2)],
                  [0,6*E*I[i]/(L[i]**2), 4*E*I[i]/L[i], 0, -6*E*I[i]/(L[i]**2), 2*E*I[i]/L[i] ],
                  [-E*A[i]/L[i], 0, 0, E*A[i]/L[i],0 ,0 ],
                  [0, -12*E*I[i]/(L[i]**3), -6*E*I[i]/(L[i]**2), 0,12*E*I[i]/(L[i]**3),-6*E*I[i]/(L[i]**2)],
                  [0,6*E*I[i]/(L[i]**2), 2*E*I[i]/L[i], 0,-6*E*I[i]/(L[i]**2), 4*E*I[i]/L[i] ]])
    
#8.2 Matriz de rotação 
    
    tau = np.array([[cosx[i], cosy[i], 0, 0 ,0 ,0],
                   [-cosy[i], cosx[i],0, 0, 0, 0],
                   [0,0,1,0,0,0],                     
                   [0,0,0,cosx[i], cosy[i], 0],
                   [0, 0, 0,-cosy[i], cosx[i],0],
                   [0,0,0,0,0,1]])

#8.3 Matrizes locais rotacionadas

    k_r = np.dot(np.dot(tau.T, k),tau)
           
#8.4 Alocação das matrizes locais na matriz global

    k_rG = np.zeros((ngl,ngl))

    a1 = int(IDB[0,i]-1)
    a2 = int(IDB[2,i])
    a3 = int(IDB[3,i]-1)
    a4 = int(IDB[5,i])

    k_rG[a1:a2,a1:a2] = k_r[0:3,0:3]
    k_rG[a3:a4,a1:a2] = k_r[3:6,0:3]
    k_rG[a1:a2,a3:a4] = k_r[0:3,3:6]
    k_rG[a3:a4,a3:a4] = k_r[3:6,3:6]

    K += k_rG 
    
#-------------------------------------------------------------------------     
#9. Vetor de Cargas
#------------------------------------------------------------------------- 

pu    = list(nos['Pu'])[0:nn]       # cargas horizontais nodais
pv    = list(nos['Pv'])[0:nn]       # cargas verticais nodais
pteta = list(nos['Pteta'])[0:nn]    # momentos fletores nodais

P       = [] 
n_index = np.array(n_id) -1

for n_index in range (nn):
    ui    = pu[n_index]
    P.append([ui])
    vi    = pv[n_index]
    P.append([vi])
    tetai = pteta[n_index]
    P.append([tetai])

P = np.array(P)

#-------------------------------------------------------------------------     
#10. Aplicar Condições de Contorno
#------------------------------------------------------------------------- 

#10.1 Montar array com os Graus de Liberdade Restringidos 

gl         = np.array(list(nos['u'])+list(nos['v'])+list(nos['teta']))
id_glr     = np.array(list(nos['ur'])+list(nos['vr'])+list(nos['tetar']))
glr        = np.trim_zeros(sorted(gl*id_glr))
remover_gl = np.array(glr)-1

#10.2 Deletar Linhas e Colunas restringidas da matriz K e do vetor P

Ki = np.delete(K, remover_gl,axis=0)
Kf = np.delete(Ki, remover_gl,axis=1)  
Pf = np.delete(P, remover_gl,axis=0)

#-------------------------------------------------------------------------     
#11. Calcular Deslocamentos
#------------------------------------------------------------------------- 

U = np.linalg.solve(Kf,Pf)

#-------------------------------------------------------------------------     
#12. Calcular as Solicitações
#------------------------------------------------------------------------- 

#12.1 Definir o vetor Deslocamentos U() integral como Ui()com zeros nos graus de liberdade restringidos

Ui    = U
a_glr = np.array(glr)
for i in range (len(glr)):
    U0 = np.insert(Ui,glr[i]-1,0)
    Ui = U0

#12.2 Definir o vetor de reações nodais em coordenadas locais Rn[]

Rn = []

for i in range (nb):

#12.2.1 Matriz de rigidez local da barra
   
    k = np.array([[E*A[i]/L[i], 0, 0, -E*A[i]/L[i],0 ,0 ],
                  [0, 12*E*I[i]/(L[i]**3), 6*E*I[i]/(L[i]**2), 0, -12*E*I[i]/(L[i]**3),6*E*I[i]/(L[i]**2)],
                  [0,6*E*I[i]/(L[i]**2), 4*E*I[i]/L[i], 0, -6*E*I[i]/(L[i]**2), 2*E*I[i]/L[i] ],
                  [-E*A[i]/L[i], 0, 0, E*A[i]/L[i],0 ,0 ],
                  [0, -12*E*I[i]/(L[i]**3), -6*E*I[i]/(L[i]**2), 0,12*E*I[i]/(L[i]**3),-6*E*I[i]/(L[i]**2)],
                  [0,6*E*I[i]/(L[i]**2), 2*E*I[i]/L[i], 0,-6*E*I[i]/(L[i]**2), 4*E*I[i]/L[i] ]])
    
#12.2.2 Matriz de rotação 
    
    tau = np.array([[cosx[i], cosy[i], 0, 0 ,0 ,0],
                   [-cosy[i], cosx[i],0, 0, 0, 0],
                   [0,0,1,0,0,0],                     
                   [0,0,0,cosx[i], cosy[i], 0],
                   [0, 0, 0,-cosy[i], cosx[i],0],
                   [0,0,0,0,0,1]])
    
#12.2.3 Matriz B dos elementos  

    index_noN = np.array(noN[i])- 1
    index_noF = np.array(noF[i])- 1
    glsN = a_gls[index_noN]
    glsF = a_gls[index_noF]
    B_gls = []
    B_gls.extend(glsN)
    B_gls.extend(glsF)
    B_gls = np.array(B_gls)
    B_gls_index = B_gls-1
        
    B = np.zeros((6,ngl))
    for n in range(6):        
        B[n,B_gls_index[n]] = 1.0
        
#12.2.4 Preencher o vetor de reações nodais em coordenadas locais Rn [] 

    u = B.dot(U0.T)        #vetor deslocamentos Globais do elemento
    p = k.dot(tau).dot(u)  #vetor de forças axiais dos elementos            
    Rn.append(p)

Rn = np.array(Rn)         #Forças de Engaste inicialmente consideradas deverãos ser superpostas.
Rn = Rn.T

#-------------------------------------------------------------------------     
#13. Calcular as Reações
#------------------------------------------------------------------------- 

#13.1 Somar U0 e as Forças de Engaste, nos graus de liberdade restringidos

FEu     = list(nos['FEu'])[0:nn]       # Forças de engaste do nó
FEv     = list(nos['FEv'])[0:nn]       # Forças de engaste do nó
FEteta  = list(nos['FEteta'])[0:nn]    # Forças de engaste do nó

FE      = [] 
n_index = np.array(n_id) -1

for n_index in range (nn):
    ui    = FEu[n_index]
    FE.append(ui)
    vi    = FEv[n_index]
    FE.append(vi)
    tetai = FEteta[n_index]
    FE.append(tetai)

FE = np.array(FE)                #Forças de Engaste consideradas

Pi = U0.dot(K) + FE              #vetor deslocamentos multiplicado pela Matriz Global + Forças de Engaste

g_restr = list(remover_gl)       #Graus de liberdade restringidos 
for i in range (len(remover_gl)):
    Rf = Pi[g_restr]             #Vetor Reações nos Graus de Liberdade Restringidos
    
#-------------------------------------------------------------------------     
#14. Plotagem 3D da Estrutura
#------------------------------------------------------------------------- 

XP=[]
YP=[]
ZP=[]
for i in range (nb):
    k3            = int(IDN[0,i] -1)  # Indexador da matriz IDN
    k4            = int(IDN[1,i] -1)  # Indexador da matriz IDN
    X1            = cx[k3],cx[k4]
    XP.extend([X1]) 
    Y1            = cz[k3],cz[k4] 
    YP.extend([Y1])
    Z1            = cy[k3],cy[k4] 
    ZP.extend([Z1])
    
fig = plt.figure(figsize=(6.4*4,4.8*2)) 
ax = plt.axes(projection="3d")

for i in range(nb): 
    ax.plot3D(XP[i],YP[i],ZP[i],linewidth=3,color='black')    
    ax.scatter3D(XP[i],YP[i],ZP[i],s=50,color='red',depthshade='false')    
    plt.xlim(0,14)
    plt.ylim(-1,1)
    ax.set_zlim(0,5)    
    plt.rcParams['xtick.labelsize'] = 10
    plt.rcParams['ytick.labelsize'] = 10
    
print(P)

ValueError: cannot convert float NaN to integer

In [119]:
custo = [2500,3000]
solu = {1:[1,2,3],2:[4,5,6]}

lista_nova = []


for j in range (3):
    x = [n[j] for n in solu.values()] 
    for i in range (1):
        lista_nova.extend([x[i]])

for j in range(1):
    lista_nova.extend([custo[j]])

print(lista_nova)

[1, 2, 3, 2500]


In [31]:
nos                = { 1:[0,0], 2:[5,0], 3:[3.2,2.4]}
x = [i[0] for i in nos.values()]  
y = [i[1] for i in nos.values()]  

print(x)


[0, 5, 3.2]
